<!--start-block-->
<hr style="height: 1px;">
<i>This code was authored by the 8.316 Course Team, Copyright 2023 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

# LECTURE 11



<a name='section_10_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.0 Overview of Learning Objectives</h2>


In this lecture we will explore the following objectives:

- Fitting For the Higgs Boson
- Combining p-values
- Convolutions
- Building Interpolated distributions
- Dealing with non-analytic forms
<br>
<!--end-block-->

<h3>Importing Libraries</h3>

Before beginning, run the cell below to import the relevant libraries for this notebook.

In [ ]:
#>>>RUN: L11.0-runcell01

import numpy as np
import lmfit
import matplotlib.pyplot as plt
import csv
from scipy import stats

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.


In [ ]:
#>>>RUN: L11.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_11_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.1 Fitting for the Higgs boson signal Background </h2>  



With all of these pieces together, I would like to compute the significance of the Higgs boson discovery in one of its main channels. To do this, we are going to use all of the tools that we have been going over. Let's first look at the data. For the Higgs boson data, there are 2 years of data each with 5 categories. Here is what all of them look like. 

In [ ]:
import lmfit 

def pol0(x,p0):
    pols=[p0]
    y = np.polyval(pols,x)
    return y

def pol1(x,p0,p1):
    pols=[p0,p1]
    y = np.polyval(pols,x)
    return y

def pol2(x, p0, p1,p2):
    pols=[p0,p1,p2]
    y = np.polyval(pols,x)
    return y

def pol3(x, p0, p1,p2,p3):
    pols=[p0,p1,p2,p3]
    y = np.polyval(pols,x)
    return y

def pol4(x, p0, p1,p2,p3,p4):
    pols=[p0,p1,p2,p3,p4]
    y = np.polyval(pols,x)
    return y

def pol5(x, p0, p1,p2,p3,p4,p5):
    pols=[p0,p1,p2,p3,p4,p5]
    y = np.polyval(pols,x)
    return y

def fitModel(iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(p0=0,p1=0,p2=0,p3=0,p4=0,p5=0)
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    #result = lmfit.minimize(binnedLikelihood, params, args=(iX,iY,(iY**0.5),iFunc))
    output = model.eval(params=result.params,x=iX)
    return output

import scipy.stats as stats 

def residual2(iY,iFunc,iYErr):
    residval = (iY-iFunc)
    return np.sum(residval**2)
    
def ftest(iY,iYerr,f1,f2,ndof1,ndof2):
    r1=residual2(iY,f1,iYerr)
    r2=residual2(iY,f2,iYerr)
    sigma2group=(r1-r2)/(ndof2-ndof1)
    sigma2=r2/(len(iY)-ndof2)
    return sigma2group/sigma2

def chi2(iY,iFunc,iYErr,iNDOF):
    resid = (iY-iFunc)/iYErr
    chi2value = np.sum(resid**2)
    chi2prob=1-stats.chi2.cdf(chi2value,len(iY)-iNDOF)
    print("Mean:",resid.mean(),"\tSTD:",resid.std(),"chi2 prob:",chi2prob)
    return chi2value/(len(iY)-iNDOF)


In [ ]:
def load(iLabel,iRange=False):
    x = np.array([])
    y = np.array([])
    label=iLabel
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter=' ')
        for row in plots:
            if not iRange and (float(row[1]) > 150 or float(row[1]) < 110):
                continue
            x = np.append(x,float(row[1]))
            y = np.append(y,float(row[2]))
            #add poisson uncertainties                                                                                                 
    weights = 1./y**0.5 
    return x,y,y**0.5,weights

def plot(ax,iLabel):
    x,y,y_err,weights=load(iLabel)
    #Now we plot it. 
    ax.errorbar(x,y,y_err, lw=2,fmt=".k", capsize=0,label=iLabel)
    #ax.x_label("$m_{\gamma\gamma}$")
    #ax.y_label("$N_{events}$")
    ax.legend()
    #ax.show()
    
fig, axs = plt.subplots(2, 3)
#2012 data    
plot(axs[0,0],"data/out.txt")
plot(axs[0,1],"data/out2.txt")
plot(axs[0,2],"data/out3.txt")
plot(axs[1,0],"data/out4.txt")
plot(axs[1,1],"data/out5.txt")
plt.show()

fig, axs = plt.subplots(2, 3)
#2011 data    
plot(axs[0,0],"data/out_2011.txt")
plot(axs[0,1],"data/out2_2011.txt")
plot(axs[0,2],"data/out3_2011.txt")
plot(axs[1,0],"data/out4_2011.txt")
plot(axs[1,1],"data/out5_2011.txt")

As you can see, from the above plots, there are way more points in the 2012 data. Lets take the category with the largest number of points, and perform an f-test on it, we can neglect the signal for now, but we will get back to that in a sec.  

In [ ]:
def fitAll(iLabel,iPlot=False):
    x,y,y_err,weights=load(iLabel)
    result0 = fitModel(x,y,weights,pol0)
    result1 = fitModel(x,y,weights,pol1)
    result2 = fitModel(x,y,weights,pol2)
    result3 = fitModel(x,y,weights,pol3)
    result4 = fitModel(x,y,weights,pol4)
    result5 = fitModel(x,y,weights,pol5)

    if iPlot:
        plt.errorbar(x,y,y_err, lw=2,fmt=".k", capsize=0,label="data")
        plt.plot(x,result0,label="pol0")
        plt.plot(x,result1,label="pol1")
        plt.plot(x,result2,label="pol2")
        plt.plot(x,result3,label="pol3")
        plt.plot(x,result4,label="pol4")
        plt.plot(x,result5,label="pol5")
        plt.xlabel("$m_{\gamma\gamma}$")
        plt.ylabel("$N_{events}$")
        plt.legend()
        plt.show()
    return x,y,y_err,result0,result1,result2,result3,result4,result5

def ftestAll(iLabel):
    x,y,y_err,result0,result1,result2,result3,result4,result5=fitAll(iLabel)
    f10=ftest(y,y_err,result0,result1,1,2)
    f21=ftest(y,y_err,result1,result2,2,3)
    f32=ftest(y,y_err,result2,result3,3,4)
    f43=ftest(y,y_err,result3,result4,4,5)
    f54=ftest(y,y_err,result4,result5,4,5)
    print("f 1 to 0:",1-stats.f.cdf(f10,1,len(y)-1))
    print("f 2 to 1:",1-stats.f.cdf(f21,1,len(y)-2))
    print("f 3 to 2:",1-stats.f.cdf(f32,1,len(y)-3))
    print("f 4 to 3:",1-stats.f.cdf(f43,1,len(y)-4))
    print("f 5 to 4:",1-stats.f.cdf(f54,1,len(y)-5))
    
fitAll("data/out.txt",True)
ftestAll("data/out.txt")

fitAll("data/out2.txt",True)
ftestAll("data/out2.txt")

So from this looks like a 4th order polynomial gives an f-test above roughly 5% for both the category with the largest yield and the second largest yield. This seems reaonsable for us to use as our background function. Let's proceed with a signal function. 

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.1.1 </span>

When we searched for the Higgs boson, we did this blind. What that means is we did look at the plots of the fits. However, we did do f-tests, and even $\chi^{2}$ goodness of fit tests. Knowing f-tests are good compute the $\chi^{2}$ of the best fit for the above. 



In [ ]:
def chi2testAll(iLabel):
    x,y,y_err,result0,result1,result2,result3,result4,result5=fitAll(iLabel)
    ###Now we test the chi2
    chi2(y,result0,y_err,1)


In [ ]:

def chi2testAll(iLabel):
    x,y,y_err,result0,result1,result2,result3,result4,result5=fitAll(iLabel)
    ###Now we test the chi2
    chi2(y,result0,y_err,1)
    chi2(y,result1,y_err,2)
    chi2(y,result2,y_err,3)
    chi2(y,result3,y_err,4)
    chi2(y,result4,y_err,5)
    chi2(y,result5,y_err,6)

chi2testAll("data/out.txt")

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
The chi2 is already good 
</pre>
        
**EXPLANATION:**
These sort of fits, we call **background only** all have a good quality of fit, and the f-test gives us different information. For signals like this where the background fraction is small, these sort of tests can be done by just looking to see if the chi2 background only fit is good. 
    
</div>



### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.1.2</span>

If the signal is large why would this be a form of unblinding (ie a seeing about our signal signfiicance before we look at the data)?


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
Yes
</pre>
        
**EXPLANATION:**
When you are looking for a large signal with high purity, and you are doing a blind analysis, you have to be careful to only look at the signal region or any indicator like the $\chi^{2}$ goodness of fit when you are read to start looking at the signal. 
    
</div>



<a name='section_11_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.2 Fitting for the Higgs boson signal Background </h2>  



Now, to fit a Higgs signal, what we want to do is a hypothesis test like we did above. Except now, we will cast our hypothesis, slightly differently to before. 

**Null Hypothesis** The Higgs signal has a mass of $m_{\gamma\gamma}$ at a specific $m_{0}$, and a fixed width 1.2 GeV. 

**Alternative Hypothesis** The Higgs signal is not there. 

The reason for the fixed width is that we know the Higgs width from our prediction of the Higgs (is 4 MeV), we also know the detector resolution by measuring photons in the experiment at other regions. In this case, we actually use $Z\rightarrow ee$ where the electrons are treated as photons. 

Also, in this case, we are going to fix the mass at 125 GeV, that way the only variable we are floating is the amplitude, and the significance we can quote by taking 2 $\Delta\log(\mathcal{L})$ and noting this should follow a $\chi^{2}_{1}$ distribution with 1 degree of freedom.

In [ ]:
def sigpol4(x,p0,p1,p2,p3,p4,amp,mass,sigma):
    bkg=pol4(x,p0,p1,p2,p3,p4)
    sig=amp*stats.norm.pdf(x,mass,sigma)
    return sig+bkg

def sigpol3(x,p0,p1,p2,p3,p4,amp,mass,sigma):
    bkg=pol3(x,p0,p1,p2,p3)
    sig=amp*stats.norm.pdf(x,mass,sigma)
    return sig+bkg

def sigpol5(x,p0,p1,p2,p3,p4,p5,amp,mass,sigma):
    bkg=pol5(x,p0,p1,p2,p3,p4,p5)
    sig=amp*stats.norm.pdf(x,mass,sigma)
    return sig+bkg

def fitModel(iX,iY,iWeights,iM,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(p0=0,p1=0,p2=0,p3=0,p4=0,p5=0,amp=0,mass=iM,sigma=1.2)
    try:
        p["mass"].vary=False
        p["sigma"].vary=False
    except:
      a=1
      #print("Mass and Sigma not in fit")
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    output = model.eval(params=result.params,x=iX)
    return output,result.residual

def fitSig(iLabel,iM,SBfunc,Bfunc,iPlot=False):
    x,y,y_err,weights=load(iLabel)
    resultSB,likeSB=fitModel(x,y,weights,iM,SBfunc)
    resultB, likeB =fitModel(x,y,weights,iM,Bfunc)
    if iPlot:
        plt.errorbar(x,y,y_err, lw=2,fmt=".k", capsize=0,label="data")
        plt.plot(x,resultSB,label="S+B")
        plt.plot(x,resultB, label="B")
        plt.xlabel("$m_{\gamma\gamma}$")
        plt.ylabel("$N_{events}$")
        plt.legend()
        plt.show()
    return np.sum(likeB**2)-np.sum(likeSB**2)

NLL=fitSig("data/out.txt",125,sigpol4,pol4,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out2.txt",125,sigpol4,pol4,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out3.txt",125,sigpol4,pol4,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out4.txt",125,sigpol4,pol4,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out5.txt",125,sigpol4,pol4,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

Wow, we see a fairly significant Higgs bump at 125, but lets scan the mass and make the so called p-value plot. This is just a plot of the significance as a function of mass. What we will do is move the mass distribution peform the same p-value calculation with the fixed signal, and we will make the plot. 

In [ ]:
def pvalueCalc(iLabel,pMass,iSBFunc,iBFunc):
    NLL=fitSig(iLabel,pMass,iSBFunc,iBFunc,False)
    NLLp = 1-stats.chi2.cdf(NLL,1)
    return NLLp

def pvaluePlot(iLabel,iSBFunc,iBFunc):
    pvalue = np.array([])
    massrange=np.linspace(110,150,120)
    for pMass in massrange:
        pvalue = np.append(pvalue,pvalueCalc(iLabel,pMass,iSBFunc,iBFunc))
    return massrange,pvalue

m0,p0 = pvaluePlot("data/out.txt",sigpol4,pol4)
m1,p1 = pvaluePlot("data/out2.txt",sigpol4,pol4)
m2,p2 = pvaluePlot("data/out3.txt",sigpol4,pol4)
m3,p3 = pvaluePlot("data/out4.txt",sigpol4,pol4)
m4,p4 = pvaluePlot("data/out5.txt",sigpol4,pol4)

plt.plot(m0,p0,label="Category 1")
plt.plot(m1,p1,label="Category 2")
plt.plot(m2,p2,label="Category 3")
plt.plot(m3,p3,label="Category 4")
plt.plot(m4,p4,label="Category 5")
plt.ylim((0.0001,1))
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("p-value")
plt.yscale("log")
plt.legend()
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.2.1</span>


Compute the Higgs boson p-value signficance plot for category 1 with a 5th order polynomial? How does it compare. 

In [ ]:
def sigpol5(x,p0,p1,p2,p3,p4,p5,amp,mass,sigma):
    sig=#
    bkg=#
    return sig+bkg

m0,p0 = pvaluePlot("out.txt",sigpol5,pol5)
m1,p1 = pvaluePlot("out2.txt",sigpol5,pol5)
m2,p2 = pvaluePlot("out3.txt",sigpol5,pol5)
m3,p3 = pvaluePlot("out4.txt",sigpol5,pol5)
m4,p4 = pvaluePlot("out5.txt",sigpol5,pol5)

plt.plot(m0,p0,label="Category 1")
plt.plot(m1,p1,label="Category 2")
plt.plot(m2,p2,label="Category 3")
plt.plot(m3,p3,label="Category 4")
plt.plot(m4,p4,label="Category 5")
plt.ylim((0.0001,1))
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("p-value")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
def pol5(x, p0, p1,p2,p3,p4,p5):
    pols=[p0,p1,p2,p3,p4,p5]
    y = np.polyval(pols,x)
    return y

def sigpol5(x,p0,p1,p2,p3,p4,p5,amp,mass,sigma):
    bkg=pol5(x,p0,p1,p2,p3,p4,p5)
    sig=amp*stats.norm.pdf(x,mass,sigma)
    return sig+bkg


#answer
NLL=fitSig("data/out.txt",125,sigpol5,pol5,True)

m03,p03 = pvaluePlot("data/out.txt",sigpol5,pol5)
plt.plot(m0,p0,label="Category 1 4th order")
plt.plot(m03,p03,label="Category 1 5th order")
plt.ylabel("p-value")
plt.yscale("log")
plt.legend()
plt.show()

#Now just for fun, lets run the full p-value plot
m0,p0 = pvaluePlot("data/out.txt",sigpol5,pol5)
m1,p1 = pvaluePlot("data/out2.txt",sigpol5,pol5)
m2,p2 = pvaluePlot("data/out3.txt",sigpol5,pol5)
m3,p3 = pvaluePlot("data/out4.txt",sigpol5,pol5)
m4,p4 = pvaluePlot("data/out5.txt",sigpol5,pol5)

plt.plot(m0,p0,label="Category 1")
plt.plot(m1,p1,label="Category 2")
plt.plot(m2,p2,label="Category 3")
plt.plot(m3,p3,label="Category 4")
plt.plot(m4,p4,label="Category 5")
plt.ylim((0.0001,1))
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("p-value")
plt.yscale("log")
plt.legend()
plt.show()


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
Less Significant
</pre>
        
**EXPLANATION:**
When you fit with more degrees of freedom, you start to lose sensitivity. The choice of freedom of the function is critical to ensure good sensitivity. 
    
</div>


<a name='section_11_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.3 Combining p-values</h2>  


Now, if we have 5 experiments each giving a p-value at a specific mass point. How, do we combine these p-values. The strategy is to realize these are each independent experiments. Moreover, it can be shown that if you have a flat prior in probability, and if you take the log of this prior and multiply by 2 $2\log(p)$ this distribution is approximately that of a $\chi^{2}$ distribution of 2 degrees of freedom.  That is to say if you have N categories each with equal sensitivity to a signal (flat prior in probability to be in any category), the the sum of the $2\log(p)$ of the categories gives us a $\chi^{2}$ distribution. 


We can see this analytically by noting that $\chi^{2}$ distribution is given by 

\begin{equation}
\chi^{2}(x,\nu) = \frac{1}{2^{\nu/2}\Gamma(\nu/2)} x^{\nu/2-1}e^{-x/2}
\end{equation}

For $\nu=2$, we have it is just a an exponential distribution given by:

\begin{equation}
\chi^{2}(x,\nu=2) = \frac{1}{2}e^{-x/2}
\end{equation}

Now for a distribution that is flat is distribution from 0 to 1, such as the $p-value$ of a random measurement. Then if we take the log of that, we find
\begin{equation}
 y = -2\log(x)\rightarrow e^{-\frac{y}{2}}=x \\
 dx = -\frac{1}{2}e^{\frac{-y}{2}} dy \\
\end{equation}
To equate probability distributions, we want to solve for scenario where the probabilities over a range are equal, namely, to get it as a function of $y$, we can write, noting $p(x)=1$
\begin{equation}
 p(y) dy = p(x) dx \\
 p(y) = p(x) \frac{dx}{dy}\\
 p(y) = p(f^{-1}(y)) \frac{dx}{dy}\\
 p(y) = \frac{dx}{dy} \\
 p(y)=e^{-\frac{y}{2}} \\
\end{equation}
or for $x$ a flat probability distribution from 0 to 1, we have that $y$ has to be distributed such that $p(y)=e^{-\frac{y}{2}}$ 

In [ ]:
xvals=np.arange(0,18,0.01)
samples = np.random.uniform(0,1,10000)
plt.hist(-2.*np.log(samples),bins=30,density=True,label='y=log(x) $x\in[0,1]$')
plt.plot(xvals,stats.chi2.pdf(xvals,2),label='$\chi^{2}_{2}(x)$')
plt.xlabel("y=log(x)")
plt.ylabel("N")
plt.legend()
plt.show()

Now going back to our calculation, we can immediately relate 
\begin{equation}
\chi^{2}_{\nu=2} = -2 \log(p_{i})
\end{equation}
Now, lets say we have $n$ measurements each with probability $p_{i}$ for the i-th category. If we take the $2\log(p_{i})$ and sum the distributions, we have a sum of $\chi^{2}$ distributions of 2 degrees of freedom. This is just a $\chi^{2}_{\nu=2n}$ distribution. 

\begin{equation}
\chi^{2}_{\nu=2n} = -2 \sum_{i=1}^{n} \log(p_{i})
\end{equation}

From this relation, we can immediately get the combined p-value by checking up the p-value of a $\chi^{2}_{\nu=2n}$ distribution.  Lets see this in action!

In [ ]:
def pvalueCalc(iLabel,pMass,iSBFunc,iBFunc):
    logp=0
    for pLabel in iLabel:
        NLL=fitSig(pLabel,pMass,iSBFunc,iBFunc,False)
        NLLp = 1.-stats.chi2.cdf(NLL,1)
        logp = logp - 2.*np.log(NLLp)
    pPVal  = 1-stats.chi2.cdf(logp,2*len(iLabel))
    return pPVal

files=["data/out.txt","data/out2.txt","data/out3.txt","data/out4.txt","data/out5.txt"]
mC,pC = pvaluePlot(files,sigpol4,pol4)

for pVal in range(4):
    sigmas = 1-stats.norm.cdf(pVal+1)
    plt.axhline(y=sigmas, color='r', linestyle='-')
plt.plot(m0,p0,label="Category 1")
plt.plot(m1,p1,label="Category 2")
plt.plot(m2,p2,label="Category 3")
plt.plot(m3,p3,label="Category 4")
plt.plot(m4,p4,label="Category 5")
plt.plot(mC,pC,label="Combined Category")
plt.ylim((0.0001,1))
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("p-value")
plt.yscale("log")
plt.legend()
plt.show()


In [ ]:
plt.plot(mC,pC,label="Category 1")
plt.ylim((0.0001,1))
plt.yscale("log")

Ok, now that we have done that, lets compare our result for the 8TeV measurement in the original paper [here](https://arxiv.org/pdf/1407.0558.pdf). If you look at that plot, you will see that there is a significance of almost 4 standard deviations for the 8TeV data, whereas we only have 3 standard deviations. The reason is that the analysis in the paper is more complicated. To compare our result with something closer, look at the Higgs discovery [paper](https://arxiv.org/pdf/1207.7235.pdf). Here, we are more sensitive, but we also have more data. 

The way the analysis is more complicated is because of the fact that the combination we are doing here is naive. We are assuming all the categories contribute equally. In reality, we know that from simulation of the signal that the relative contributions of each category is not flat, but weighted. As a consequnce, the trick in this section really only works to get an "approximate" sensitivity. The real way to do this is to simultaneously fit all categories with the known, relative signal strength of each of the categories. 

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.3.1</span>

Noting that the p-value on the y-axis is just a translation of the likelihood, and the p-value is computed from a $\chi^{2}$ distribution, convert from $\chi^{2}$ probability back to 2$\log(\mathcal{L})$, from this compute the best fit mass for the Higgs boson? How does it compare to the true value



In [ ]:
#Answer, the strategy here is to realize that p-value plot is also 2*Log(L) of our best fit, thus, we just need to go 1 standard deviation from the minimum in likelihood
logC = stats.chi2.ppf(1-pC,1)
logC = -logC+np.max(logC)
plt.plot(mC,logC,label="Combined")
plt.ylim(0,2)
plt.xlim(124,126)
#plt.hlines(1,120,130)
plt.xlabel("m$_{\gamma\gamma}$ (GeV)")
plt.ylabel("2$\Delta$log(L) ")
plt.show()
#Its roughly 124.8-125.5 => 125.2+/-0.35 (Its almost spot on!)

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
Our best fit mass is from 124.8 to 125.5 => 125.2 +/- 0.35 (almost spont on!)
</pre>
        
**EXPLANATION:**
We can obtain this by noting that the p-value is jsut a translation of the minimum likelihood. So we just need to convert from p-value back to likelihood and scan 2$\log(\mathcal{L})$, and viola. 
    
</div>


<a name='section_10_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.4 Function choice </h2>  

Now, we chose a background function based on a [power law](https://en.wikipedia.org/wiki/Power_law) others you can consider are the [laurent polynomials](https://en.wikipedia.org/wiki/Laurent_polynomial). This will just be taking $x^{-j}$ instead of $x^{j}$. The point for this is that the choice that we made to fit the function had some level of aribratriness. What would our fit performance be if we just chose another fit function somewhat at ranodm? In fact, more generally, if we have another fit function that works, which one is best? 

This section we will try to address this arbitrariness by noting how these fits behave in general. Let's take a deeper dive into what is going on. we are goin to make a set of progressive more complciated functions


In [ ]:
def bkg1(x,p0,p1):
    pols=[p0,p1]
    y=np.polyval(pols,x)
    #y=np.exp(-p0*(x-p1))
    return y

def bkg2(x, p0,p1,p2):
    #pols=[p0,p1,p2]
    #y=np.polyval(pols,x)
    y=p0*np.exp(-p1*(x-p2))
    return y

def bkg3(x, p0, p1,p2,p3):
    #pols=[p0,p1,p2,p3]
    y=p0*np.exp(-p1*(x-p2))+p3
    return y

def bkg4(x, p0, p1,p2,p3,p4):
    pols=[p3,p4]
    y=p0*np.exp(-p1*(x-p2))+np.polyval(pols,x)
    return y

def bkg5(x, p0, p1,p2,p3,p4,p5):
    pols=[p3,p4,p5]
    y=p0*np.exp(-p1*(x-p2))+np.polyval(pols,x)
    return y

def bkg6(x, p0, p1,p2,p3,p4,p5,p6):
    pols=[p3,p4,p5,p6]
    y=p0*np.exp(-p1*(x-p2))+np.polyval(pols,x)
    return y

def fitModel(iX,iY,iWeights,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(p0=0,p1=0.05,p2=100,p3=0.05,p4=0,p5=0,p6=0)
    try:
        p['p1'].vary=False
    except:
        a=1
    try:
        p['p2'].vary=False
    except:
        a=1
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    #lmfit.report_fit(result)
    output = model.eval(params=result.params,x=iX)
    return output

def fitAll(iLabel,iPlot=False):
    x,y,y_err,weights=load(iLabel)
    result0 = fitModel(x,y,weights,pol0)
    result1 = fitModel(x,y,weights,bkg1)
    result2 = fitModel(x,y,weights,bkg2)
    result3 = fitModel(x,y,weights,bkg3)
    result4 = fitModel(x,y,weights,bkg4)
    result5 = fitModel(x,y,weights,bkg5)
    result6 = fitModel(x,y,weights,bkg6)
    
    f65=ftest(y,y_err,result5,result6,4,5)
    print("f 6 to 5:",1-stats.f.cdf(f65,1,len(y)-1))
    if iPlot:
        plt.errorbar(x,y,y_err, lw=2,fmt=".k", capsize=0,label="data")
        plt.plot(x,result0,label="bkg0")
        plt.plot(x,result1,label="bkg1")
        plt.plot(x,result2,label="bkg2")
        plt.plot(x,result3,label="bkg3")
        plt.plot(x,result4,label="bkg4")
        plt.plot(x,result5,label="bkg5")
        plt.plot(x,result6,label="bkg5")
        plt.xlabel("$m_{\gamma\gamma}$")
        plt.ylabel("$N_{events}$")
        plt.legend()
        plt.show()
    return x,y,y_err,result0,result1,result2,result3,result4,result5

#just showing 4 since its the hardest to fit
fitAll("data/out.txt",True)
ftestAll("data/out.txt")
chi2testAll("data/out.txt")



So from above, a the second order constant + power law and a linear term is sufficient to fit he data both with a good $\chi^{2}$ and a good f-test. Notice the f-test is not perfect higher orders. In any case, lets go ahaed and chose this function, and see how this changes our significance? 

In [ ]:
def fitModel(iX,iY,iWeights,iM,iFunc):
    model  = lmfit.Model(iFunc)
    p = model.make_params(p0=0,p1=0,p2=0,p3=0,p4=0,p5=0,amp=0,mass=iM,sigma=1.2)
    try:
        p["mass"].vary=False
        p["sigma"].vary=False
    except:
      a=1
      #print("Mass and Sigma not in fit")
    result = model.fit(data=iY,params=p,x=iX,weights=iWeights)
    output = model.eval(params=result.params,x=iX)
    return output,result.residual

def sigbkg5(x,p0,p1,p2,p3,p4,p5,amp,mass,sigma):
    bkg=bkg5(x,p0,p1,p2,p3,p4,p5)
    sig=amp*stats.norm.pdf(x,mass,sigma)
    return sig+bkg

#answer
NLL=fitSig("data/out.txt",125,sigbkg5,bkg5,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out2.txt",125,sigbkg5,bkg5,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out3.txt",125,sigbkg5,bkg5,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out4.txt",125,sigbkg5,bkg5,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))

NLL=fitSig("data/out5.txt",125,sigbkg5,bkg5,True)
print("2NLL:",NLL,"p-value",1-stats.chi2.cdf(NLL,1))


Our p-value for this one category went down from 0.004 to 0.048. This means we are getting less excess. Could this mean we are more sensitive? Let's do the full combined p-value scan with this assumptions, and compare.  Note that the scan below now starts to take a little bit of time (few min). 

While you are waiting to run the code, maybe you can start to guess which function will be more sensitive? 

Moreover, given that, is there anything we gain from having multiple functions? 

In [ ]:
def pvaluePlot(iLabel,iSBFunc,iBFunc):
    pvalue = np.array([])
    massrange=np.linspace(120,130,10)
    for pMass in massrange:
        print("Mass:",pMass)
        pvalue = np.append(pvalue,pvalueCalc(iLabel,pMass,iSBFunc,iBFunc))
    return massrange,pvalue


files=["data/out.txt","data/out2.txt","data/out3.txt","data/out4.txt","data/out5.txt"]
mCB2,pCB2 = pvaluePlot(files,sigbkg5,bkg5)

for pVal in range(4):
    sigmas = 1-stats.norm.cdf(pVal+1)
    plt.axhline(y=sigmas, color='r', linestyle='-')
plt.plot(mC,pC,label="Combined Category (polynomial)")
plt.plot(mCB2,pCB2,label="Combined Category (Power law)")
plt.ylim((0.0001,1))
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("p-value")
plt.yscale("log")
plt.legend()
plt.show()


Now, in light of this, there has been a recent development known as discrete profiling that has allowed to extend the scope of our interpretations. The idea here is that if we account for our degrees of freedom to be different, we can profile a whole library of functions simultaneously, and we can compare all of them by looking at their corrected likelihood. 

To do this, we are going to do something a little ad-hoc here and just look at one explempary category so that we can really understand how the envelope changes our measurements. What we will do below, is replace our p-value with a likelihood scan, then we will scan around the Higgs mass and show the likelihood

In [ ]:
def logCalc(iLabel,pMass,iSBFunc,iBFunc,iNDOF):
    lNLL=iNDOF
    for pLabel in iLabel:
        pNLL=fitSig(pLabel,pMass,iSBFunc,iBFunc,False)
        lNLL -= pNLL
    return lNLL

def NLLPlot(iLabel,iSBFunc,iBFunc,iNDOF):
    NLL = np.array([])
    massrange=np.linspace(123,127,40)
    for pMass in massrange:
        NLL = np.append(NLL,logCalc(iLabel,pMass,iSBFunc,iBFunc,iNDOF))
    return massrange,NLL

files=["data/out2.txt"]
mCP2,lLLP = NLLPlot(files,sigpol4,pol4,5)
mCB2,lLLB = NLLPlot(files,sigbkg5,bkg5,4)

plt.plot(mCP2,lLLP,label="Combined Category (polynomial)")
plt.plot(mCB2,lLLB,label="Combined Category (Power law)")
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$-2\Delta LL$")
plt.legend()
plt.show()


The true minimum is defined by the envelope in the corrected likelihood, that really gives us the full information that enables us to quote a result. Discrete profiling thus enables us to get even more sensitive if our envelope narrows. However, its often that it widens, this is the uncertainty coming in from function choice. The point being is that by being more expressive by adding more function choices, we naturally give the fit more flexibility to fit well. If there are more options that fit well, then our uncertainties get larger.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.4.1</span>

Based on the above envelope compute the Higgs mass? 


In [ ]:
minall=np.minimum(lLLB.flatten(),lLLP.flatten())
minall=np.min(minall)
plt.plot(mCB2,np.minimum(lLLB,lLLP)-minall,label="Min LL")
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$-2\Delta LL$")
plt.legend()
plt.show()

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
Our best fit mass is from 124.4 to 125.3 => 124.8 +/- 0.35 (almost spont on!)
</pre>
        
**EXPLANATION:**
The minimum is not now a quadratic and has a bulge on the left. This shows we have added a systematic for function choice and made the uncertainties slightly larger. 
    
</div>


<a name='section_10_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.5 Convolutions</h2>  



Convlutions are a critical component of evey good statistical analysis. Its a way to multiply distributions together. 
Lets build convlutions up by scratch and then go from there. 

The core concept of a convolution is that you are effectively multiplying distributions. Given two functions $f(x)$ and $g(x)$, we can define convolutions by 
\begin{eqnarray}
(f*g)(z) &=& \int^{\infty}_{-\infty} f(z-t)g(t)dt
\end{eqnarray}
For data analysis, we usually think about this in the context of probability distribuitons $g$ and $f$. From here, we construct a new probability distribution $(f*g)$. Anyway, lets take a look at how it works. 

![alternative text](convolution.gif)

To do this, lets first define some functions to convolve. 

In [ ]:
#First lets define a triangular distribution
def triangle(x,mean=5):
    Norm=mean*mean
    val=np.where(x <= mean,np.maximum(x,np.zeros(len(x))), np.maximum(2*mean-x,np.zeros(len(x))))
    return val/Norm

#Now define the gaussian
def gaussian(x,mean=0,sigma=1):
    return 1./(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mean)**2 / (2 * sigma**2)) 


In [ ]:
#Now lets do a convolution by hand
def convolve(f1,f2,x,sigma=1,iMin=-10,iMax=10,iN=2000):
    step=(iMax-iMin)/iN
    pInt=0
    for i0 in range(iN):
            pX   = np.repeat(i0*step+iMin,len(x))
            pVal = f1(x-pX,sigma=sigma)*f2(pX)
            pInt += pVal*step
    return pInt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
tri_out=triangle(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,triangle,x_in)
conv2_out=convolve(gaussian,triangle,x_in,sigma=2)

ax.plot(x_in,tri_out,label='triangle')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

So why are these convolutions so important? 

We can get a picture of how important these are 

![alternative text](gauss_blur.png)

Effectively, when we have poor resolution of things coming from an uncertainty in how we measure something. Distributions, which are originally nice and sharp can get smeared out. The easy way to describe all of these is through a convolution. There is a rich literature to discribe convolutions. 

A critical element to remember about the convolutions is that in Fourier space convolutions can be performed as straight multiplications and then inversing the fourier transform.  


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.5.1</span>

What happens if you convolve a Gaussian with a straight line: $y=0$? You can use the code from the previous cell to plot it.


In [ ]:
#>>>SOLUTION

def line(x):
  return x 

fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
line_out=line(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,line,x_in)
conv2_out=convolve(gaussian,line,x_in,sigma=5)

ax.plot(x_in,line_out,label='line')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()


<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
See code above
</pre>
        
**EXPLANATION:**
    
The convolution will stretch out the line by weighting points with a gaussian. The stretching only occurs on the boundary where the convolution is underway. As a result, the core line doesn't actually change in shape. 
    
</div>

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.5.2</span>

What happens if you convolve a Gaussian with a Gaussian? (you can again use the code before it)

In [ ]:
#>>>SOLUTION


fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
line_out=line(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,gaussian,x_in)
conv2_out=convolve(gaussian,gaussian,x_in,sigma=5)

ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
See code above
</pre>
        
**EXPLANATION:**
    
A Gauassian convolved with a gaussian just gives you another Gaussian, you can prove this by fourier transforming the convolution and noting that according to the [Convolution theorem](https://en.wikipedia.org/wiki/Convolution_theorem) a convolution is just equal to a product of fourier transforms. 

</div>

### challenge question

What does $f(x)=\sin(x)$ convoled with a gaussian look like? How does it change by the resolution of the gaussian?

In [ ]:
def func(x):
    return np.sin(x)*x

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
func_out=func(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,func,x_in)
conv2_out=convolve(gaussian,func,x_in,sigma=2)

ax.plot(x_in,func_out,label='sin(x)')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=2$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

In [ ]:
#answer
def func(x):
    return np.sin(x)*x

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
sin_out=sin(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,sin,x_in,sigma=2)


ax.plot(x_in,sin_out,label='triangle')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.5.3</span>

Often we use convolutions to "Smear" our distributions analytically without having to build a simulation. In the code below we sample points from a gaussian. Take the original function below ( a box disribution), add points sampled by a gaussian, and show that the distribution matches the convolution.

In [ ]:
#>>>EXERCISE
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.
lNToys=1000
nbins=100
sigma=1
smeared=np.random.normal(0,sigma,(lNToys,nbins))
x_in=np.linspace(-20, 20,(nbins))
smeared_x_in=#your code here, make 1000 example x distributiosn from above

def func(x):
    return 0+0.1*np.where(x < -10,0,1) - 0.1*np.where(x < 10,0,1)

box_out=func(x_in)
smeared_box_out=#your code here, average over 1000 sampled distributions



In [ ]:
#>>>SOLUTION
#>>>RUN
lNToys=1000
nbins=100
sigma=1
smeared=np.random.normal(0,sigma,(lNToys,nbins))
x_in=np.linspace(-20, 20,(nbins))
smeared_x_in=x_in+smeared

def func(x):
    return 0+0.1*np.where(x < -10,0,1) - 0.1*np.where(x < 10,0,1)

#now lets plot
fig, ax = plt.subplots()
x_in=np.linspace(-20, 20, 100)
box_out=func(x_in)
smeared_box_out=np.mean(func(smeared_x_in),axis=0)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,func,x_in)


ax.plot(x_in,box_out,label='box(x)')
ax.plot(x_in,smeared_box_out,label='smeared box(x)')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

<div style="border:1.5px; border-style:solid; padding: 0.5em; border-color: #90409C; color: #90409C;">

**SOLUTION:**

<pre>
See code above
</pre>
        
**EXPLANATION:**
    
This is a first demonstration of a monte carlo technique to approximate a resolution. In Monte Carlo, we randomly sample distributions and then simulate them to mimic the effect that we would like our output to look like. In this case, we are smearing our inputs and applying a function on our smeared inputs. Knowing when to integrate and when to sample is something that is very powerful as we go forward. 
    
</div>


<a name='section_11_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.6 Building Interpolated Distributions</h2>  


To find the right fit function, we used a library of functions to profile and find the minimum. Thanks to the f-statistic, we don't need to just throw one function at the problem, we can throw many. In fact, modern searches aiming for the most sensitivity will send a library of functions to fit a signal and not just one. For a more detailed analysis of how you would do this look at this [paper](https://arxiv.org/pdf/1408.6865.pdf). 

However, what we can also do is actually build functions by just throwing it to the data. To give you two ways to do this, let's first try a spline interpolated function. 

In [ ]:
#Now lets load some data and do gaussian kernals with it
x,y,y_err,weights=load("data/out_2011.txt")


from scipy import interpolate
tck = interpolate.splrep(x, y) #setup the spline
x2 = np.linspace(110, 150) #range
y2 = interpolate.splev(x2, tck)#apply the spline

plt.plot(x, y, 'go',label='data')
plt.plot(x2, y2, 'b',label='spline')
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$N_{events}$")
plt.legend()
plt.show()

So what happened is, we just took the data and turned it into a smooth function that we can evaulate anywhere. This is done by chunking up the data into little pieces and fitting higher order polynomials to each. We end up with a function. 

Since its a function, we can do whatever we want with it. Let's smooth this function out by convolving it with a Gaussian distribution. 

In [ ]:
#spline convolve by hand
def splineconvolve(tck,f2,x,iMin=-15,iMax=15,iN=500):
    step=float((iMax-iMin))/float(iN)
    pInt=0
    for i0 in range(iN):
            pX   = i0*step+iMin
            pVal = interpolate.splev(x-pX,tck)*f2(pX)
            pInt += pVal*step
    return pInt

def gaussian(x,mean=0,sigma=5):
    return 1./(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mean)**2 / (2 * sigma**2)) 

fig, ax = plt.subplots()
x_in=np.linspace(122, 142, 10)
conv_out=[]
for val in x_in:
    pConv_out=splineconvolve(tck,gaussian,val)
    conv_out.append(pConv_out)

#now we cna plot it
plt.plot(x, y, 'go',label='data')
plt.plot(x2, y2, 'b',label='spline')
plt.plot(x_in,conv_out,c='orange',label='convolution')
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$N_{events}$")
plt.legend()
plt.show()

In fact, in recent times interpolation with all sorts of functions, not just polynomials have become very popular. Just to show you another one that is used often, lets try Gaussian processes. Gaussian processes, aim to fit guassian distributiosn to describe the data. The strategy, conceptually, is like the f-test. Keep adding Gaussian's to fit the data until it is well described. See [here](https://en.wikipedia.org/wiki/Gaussian_process) for a much deeper explanation for how this would work. 

In [ ]:
!pip install george
import george
from george import kernels

kernel = np.var(y) * kernels.Matern52Kernel(5.0)
#kernel = np.var(y) * kernels.Matern52Kernel(125.0)
gp = george.GP(kernel)
gp.compute(x, y_err)
x_pred = np.linspace(110, 150, 100)
pred, pred_var = gp.predict(y, x_pred, return_var=True)

plt.fill_between(x_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),color="k", alpha=0.2,label="gaussian Process")
plt.plot(x_pred, pred, "k", lw=1.5, alpha=0.5)
plt.errorbar(x, y, yerr=y_err, fmt=".k", capsize=0)
plt.plot(x2, y2, 'b',label='spline')
plt.plot(x_in,conv_out,c='orange',label='convolution')
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$N_{events}$")
plt.legend()
plt.show()

As with the spline, there are parameters for how it averages over the points. Try the kernal 125 above, you will see that this is is effectively changing the size of the window of interpolation. This will allow us to either smooth or unsmooth our distribution.  

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-11.6.1</span>

When we we have a spline function, we can now do coordinate tansforms of whole distributions, take the above spline, shift it by a 5 GeV and plot it again? 

In [ ]:
fig, ax = plt.subplots()

xs  = ###Your code here
ys  = ###Your code here

#now we cna plot it
plt.plot(x, y, 'go',label='data')
plt.plot(x2, y2, 'r',label='spline')
plt.plot(xs, ys, 'b',label='spline shift')
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$N_{events}$")
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

xs  = np.linspace(110,145,100)
ys  = interpolate.splev(xs+5, tck)#apply the spline

#now we cna plot it
plt.plot(x, y, 'go',label='data')
plt.plot(x2, y2, 'r',label='spline')
plt.plot(xs, ys, 'b',label='spline shift')
plt.xlabel("$m_{\gamma\gamma}$")
plt.ylabel("$N_{events}$")
plt.legend()
plt.show()

<a name='section_10_8'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L11.7 Dealing with non-analytic forms</h2>  



At this point, we have gone a bit away from just fitting functions. Its not always the case that we have a nice function that we can fit to describe our data. It is sometimes the case that we have a simulated shape. For High energy physics, simulated shapes often come from so called "Monte-Carlo" simulation. In this approach, we construct distributions by randomly sampling many millions of events. The resulting simulated events can then be treated like data. The data set below represents another Higgs boson channel, the Higgs decay to 4 leptons. However, in this instance, we want to fit the peak at 90 with a simulation of the peak. 

In [ ]:
x,y_data,y_err,weights=load("data/data.txt",True)
x,y_mc,y_mc_err,_=load("data/zz_narrow.txt",True)

tck = interpolate.splrep(x, y_mc)
x2 = np.linspace(50, 160,1000)
y2 = interpolate.splev(x2, tck)
plt.errorbar(x,y_data,yerr=y_err,marker='.',linestyle = 'None', color = 'black')
plt.plot(x2, y2, 'b')
plt.plot(x,y_mc,drawstyle = 'steps-mid')
plt.xlabel("$m_{4\ell}$")
plt.ylabel("$N_{events}$")
plt.legend()
plt.show()

There are clearly a few things off. First of all the number of events is off. Secondly, the shapes don't look exactly the same. 

Oftentimes, when we actually want to do a precision fit, we will rely on our simulated samples to extract the signal. What we will do is allow the shape to be modified by a number of different approaches. One appraoch is to apply a numerical convolution of the shape with a gaussian distribution, so that we can smear it out, making it wider. This what we will do here. 

In [ ]:
#spline convolve by hand
def splineconvolvegaus(x,mean,sigma,iMin=-15,iMax=15,iN=500):
    step=float((iMax-iMin))/float(iN)
    pInt=0
    for i0 in range(iN):
            pX   = i0*step+iMin
            pVal = interpolate.splev(x-pX,tck)*gaussian(pX,mean,sigma)
            pInt += pVal*step
    return pInt

def gausconv(x,mean,sigma,amp,a,b):
    val=splineconvolvegaus(x,mean,sigma)*amp
    val=a + b*x + val
    return val

model  = lmfit.Model(gausconv)
p = model.make_params(mean=0,sigma=1.0,amp=3.0,a=1,b=0)
p["sigma"].value=0.1
#p["sigma"].vary=False
result = model.fit(data=y_data,params=p,x=x,weights=weights)
lmfit.report_fit(result)
result.plot()

This doesn't fully get the data correct, but its definitely a big step towards getting something much better. You will have to use these distributions in future datasets, so you will see more about how things. Just to get a better idea of what is going in this fit, lets actually look at what happens, if we don't allow the peak to move aroundn, and we also fix the resolution to be narrow. 

In [ ]:
def splineconvolvegaus(tck,f2,x,mean,sigma,iMin=-15,iMax=15,iN=500):
    step=float((iMax-iMin))/float(iN)
    pInt=0
    for i0 in range(iN):
            pX   = i0*step+iMin
            pVal = interpolate.splev(x-pX,tck)*f2(pX,mean,sigma)
            pInt += pVal*step
    return pInt

def gausconv(x,mean,sigma,sig,baseline,slope):
    val=splineconvolvegaus(tck,gaussian,x,mean,sigma)
    output = baseline+sig*val+slope*x
    return output

model  = lmfit.Model(gausconv)
p = model.make_params(mean=0,sigma=1,sig=2,baseline=2,slope=0)
p["mean"].vary = False
p["sigma"].vary = True
result = model.fit(data=y_data, params=p, x=x, weights=weights)
lmfit.report_fit(result)
result.plot()

#Now lets not smear the data
p["sigma"].value = 0.01
p["sigma"].vary = False
result = model.fit(data=y_data, params=p, x=x, weights=weights)
lmfit.report_fit(result)
_=result.plot()

##  Going beyond

This is just the start of construction fit functions. The next set of tools we can explore is how to build deep learning algorithms to model the data. As we have gone through this lecture, and the previous lectures, we have been abstracting and standardizing the tools more and more to get to the point of automating the whole procedure that is the point and this is where we start for the next sections of the course. 